In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px

/Users/fklein/neuefische/capstone/Capstone_WindPowerPredicting/.venv/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
from io import SEEK_CUR
import pandas as pd
import numpy as np

import dash
from datetime import date
import dash_bootstrap_components as dbc
from dash import dcc
from dash import html
import plotly.graph_objects as go
import plotly.express as px

from PIL import ImageColor

from dash.dependencies import Input, Output, State

from dashboard_main import *

In [3]:

features_forecast = ['ZONEID', 'TIMESTAMP', 'HOUR', 'WEEKDAY', 'WS100', 'WD100']
features_retro = ['ZONEID', 'TIMESTAMP', 'HOUR', 'WEEKDAY', 'TARGETVAR']

PATH_PREDICTIONS = 'RandomForest_Predictions.csv'
PATH_DATA_ALL = 'raw_data_incl_features_test.csv'

initial_date = '2013-07-01'

In [4]:
data_forecast = pd.read_csv(PATH_PREDICTIONS, parse_dates=['TIMESTAMP'])
data_forecast.rename(
    columns={x: 'Wind Farm '+x.split()[-1] 
    for x in data_forecast.columns if x.startswith('Zone')}, 
    inplace=True)
add_HOUR_column(data_forecast)
data_all = pd.read_csv(PATH_DATA_ALL, parse_dates=['TIMESTAMP'])
data_all.interpolate(method='bfill', inplace=True)
data_all['ZONEID'] = data_all['ZONEID'].apply(lambda x: 'Wind Farm '+str(x))
data_all['HOUR'].replace(0,24, inplace=True)
data_all.replace('Zone', 'Wind Farm', inplace=True)
data_all.reset_index(inplace=True)

In [5]:
day = '2013-08-15'
zone = 'Wind Farm 8'

In [6]:
delta_minute = pd.Timedelta(1, unit='min')

In [7]:
data_forecast['TIMESTAMP'] = data_forecast['TIMESTAMP'].apply(lambda x: x-delta_minute)
data_forecast.drop('HOUR', axis=1, inplace=True)

In [8]:
data_forecast = data_forecast[['TIMESTAMP', zone]]

In [9]:
data_forecast['DATE'] = data_forecast['TIMESTAMP'].dt.date 

In [10]:
data_forecast = data_forecast.groupby('DATE').mean()

In [11]:
data_forecast_mean = data_forecast.mean(axis=1)
data_forecast_mean
df_forecast_mean = pd.DataFrame(data_forecast_mean)

In [12]:
df_forecast_mean.columns = ['TARGETVAR_PREDICTED']
df_forecast_mean

,TARGETVAR_PREDICTED
DATE,
2013-07-01,0.600115
2013-07-02,0.563627
2013-07-03,0.584431
2013-07-04,0.732470
2013-07-05,0.724957
...,...
2013-12-27,0.299677
2013-12-28,0.380428
2013-12-29,0.091375


## observed values

In [13]:
data_all

,index,ZONEID,TIMESTAMP,TARGETVAR,U10,V10,U100,V100,HOUR,MONTH,WEEKDAY,IS_HOLIDAY,WS10,WS100,WD10,WD100,WD100CARD,WD10CARD,U100NORM,V100NORM
0,0,Wind Farm 1,2013-06-30 01:00:00,0.375811,4.519248,-1.559637,6.913626,-2.293687,1,6,6,0,4.780803,7.284176,289.040056,288.353946,WNW,WNW,0.949129,-0.314886
1,1,Wind Farm 1,2013-06-30 02:00:00,0.519688,4.810528,-1.128186,6.989617,-1.628825,2,6,6,0,4.941051,7.176895,283.198721,283.117810,WNW,WNW,0.973905,-0.226954
2,2,Wind Farm 1,2013-06-30 03:00:00,0.520346,4.560787,-0.831092,6.590505,-1.121018,3,6,6,0,4.635892,6.685165,280.327442,279.653384,W,W,0.985840,-0.167687
3,3,Wind Farm 1,2013-06-30 04:00:00,0.428907,4.875876,-0.787637,7.088597,-1.173369,4,6,6,0,4.939083,7.185054,279.176149,279.398894,W,W,0.986575,-0.163307
4,4,Wind Farm 1,2013-06-30 05:00:00,0.665915,4.956422,-0.793845,7.314015,-1.150858,5,6,6,0,5.019592,7.404005,279.099489,278.942154,W,W,0.987846,-0.155437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44395,44395,Wind Farm 10,2013-12-31 20:00:00,NaN,1.032363,-6.281558,2.041033,-11.220655,20,12,1,0,6.365826,11.404776,350.666975,349.690637,N,N,0.178963,-0.983856
44396,44396,Wind Farm 10,2013-12-31 21:00:00,NaN,1.702361,-6.202448,2.846245,-10.486079,21,12,1,0,6.431827,10.865494,344.652195,344.814040,NNW,NNW,0.261953,-0.965081
44397,44397,Wind Farm 10,2013-12-31 22:00:00,NaN,5.086629,-1.261378,7.382256,-3.097656,22,12,1,0,5.240694,8.005821,283.927218,292.763328,WNW,WNW,0.922111,-0.386925
44398,44398,Wind Farm 10,2013-12-31 23:00:00,NaN,4.183751,-1.580172,5.789054,-2.116548,23,12,1,0,4.472216,6.163840,290.691183,290.083020,WNW,WNW,0.939196,-0.343381


In [14]:
# data_all = data_all[data_all['ZONEID'] == zone]
data_all

,index,ZONEID,TIMESTAMP,TARGETVAR,U10,V10,U100,V100,HOUR,MONTH,WEEKDAY,IS_HOLIDAY,WS10,WS100,WD10,WD100,WD100CARD,WD10CARD,U100NORM,V100NORM
0,0,Wind Farm 1,2013-06-30 01:00:00,0.375811,4.519248,-1.559637,6.913626,-2.293687,1,6,6,0,4.780803,7.284176,289.040056,288.353946,WNW,WNW,0.949129,-0.314886
1,1,Wind Farm 1,2013-06-30 02:00:00,0.519688,4.810528,-1.128186,6.989617,-1.628825,2,6,6,0,4.941051,7.176895,283.198721,283.117810,WNW,WNW,0.973905,-0.226954
2,2,Wind Farm 1,2013-06-30 03:00:00,0.520346,4.560787,-0.831092,6.590505,-1.121018,3,6,6,0,4.635892,6.685165,280.327442,279.653384,W,W,0.985840,-0.167687
3,3,Wind Farm 1,2013-06-30 04:00:00,0.428907,4.875876,-0.787637,7.088597,-1.173369,4,6,6,0,4.939083,7.185054,279.176149,279.398894,W,W,0.986575,-0.163307
4,4,Wind Farm 1,2013-06-30 05:00:00,0.665915,4.956422,-0.793845,7.314015,-1.150858,5,6,6,0,5.019592,7.404005,279.099489,278.942154,W,W,0.987846,-0.155437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44395,44395,Wind Farm 10,2013-12-31 20:00:00,NaN,1.032363,-6.281558,2.041033,-11.220655,20,12,1,0,6.365826,11.404776,350.666975,349.690637,N,N,0.178963,-0.983856
44396,44396,Wind Farm 10,2013-12-31 21:00:00,NaN,1.702361,-6.202448,2.846245,-10.486079,21,12,1,0,6.431827,10.865494,344.652195,344.814040,NNW,NNW,0.261953,-0.965081
44397,44397,Wind Farm 10,2013-12-31 22:00:00,NaN,5.086629,-1.261378,7.382256,-3.097656,22,12,1,0,5.240694,8.005821,283.927218,292.763328,WNW,WNW,0.922111,-0.386925
44398,44398,Wind Farm 10,2013-12-31 23:00:00,NaN,4.183751,-1.580172,5.789054,-2.116548,23,12,1,0,4.472216,6.163840,290.691183,290.083020,WNW,WNW,0.939196,-0.343381


In [15]:
## observed values
data_all['TIMESTAMP'] = data_all['TIMESTAMP'].apply(lambda x: x-delta_minute)

In [16]:
data_all['DATE'] = data_all['TIMESTAMP'].dt.date

In [17]:
data_all_mean = data_all.groupby('DATE').mean()['TARGETVAR']

In [18]:
df_all_mean = pd.DataFrame(data_all_mean)

In [19]:
df_all_mean.columns = ['TARGETVAR_OBSERVED']
df_all_mean

,TARGETVAR_OBSERVED
DATE,
2013-06-30,0.409286
2013-07-01,0.478447
2013-07-02,0.461625
2013-07-03,0.775218
2013-07-04,0.793554
...,...
2013-12-27,0.352287
2013-12-28,0.651090
2013-12-29,0.185031


In [20]:
df_comparison = df_all_mean.merge(df_forecast_mean, how='inner', on='DATE')

In [21]:
df_comparison.columns = ['TARGETVAR_OBSERVED', 'TARGETVAR_PREDICTED']
df_comparison

,TARGETVAR_OBSERVED,TARGETVAR_PREDICTED
DATE,,
2013-07-01,0.478447,0.600115
2013-07-02,0.461625,0.563627
2013-07-03,0.775218,0.584431
2013-07-04,0.793554,0.732470
2013-07-05,0.678502,0.724957
...,...,...
2013-12-27,0.352287,0.299677
2013-12-28,0.651090,0.380428
2013-12-29,0.185031,0.091375


In [22]:

fig = go.Figure()

fig.add_traces(
    go.Scatter(x=df_comparison.index, y = df_comparison['TARGETVAR_OBSERVED'], 
        mode = 'lines', line=dict(color='red',), name='observed',
    )
)

fig.add_traces(
    go.Scatter(x=df_comparison.index, y = df_comparison['TARGETVAR_PREDICTED'], 
        mode = 'lines', line=dict(color='blue',), name='predicted',
    )
)

In [25]:
pd.to_datetime(data_comparison['DATE'])

0      2013-07-01
1      2013-07-01
2      2013-07-01
3      2013-07-01
4      2013-07-01
          ...    
1835   2013-12-31
1836   2013-12-31
1837   2013-12-31
1838   2013-12-31
1839   2013-12-31
Name: DATE, Length: 1840, dtype: datetime64[ns]

In [38]:
## get the required data for the comparison between actual and predicted values
def get_data_up_to_day(day, 
                       data,
                       days_before=0):
            
    day +=' 0:00:00'
    time_start = pd.to_datetime(day) - pd.Timedelta(value=days_before, unit='day')
    time_end = pd.to_datetime(day) - pd.Timedelta(value=1, unit='day')
    data = data[(data['DATE']>=time_start) & (data['DATE']<=time_end)]
    return data

In [39]:
data_comparison['DATE'] = pd.to_datetime(data_comparison['DATE'])

In [40]:
get_data_up_to_day("2013-08-10", 
         data_comparison,
         days_before=10)

,DATE,ZONEID,TARGETVAR_OBSERVED,TARGETVAR_PREDICTED
300,2013-07-31,Wind Farm 1,0.172759,0.104888
301,2013-07-31,Wind Farm 10,0.038418,0.089815
302,2013-07-31,Wind Farm 2,0.101359,0.132726
303,2013-07-31,Wind Farm 3,0.020180,0.081799
304,2013-07-31,Wind Farm 4,0.069517,0.050294
...,...,...,...,...
395,2013-08-09,Wind Farm 5,0.612263,0.466864
396,2013-08-09,Wind Farm 6,0.580959,0.523162
397,2013-08-09,Wind Farm 7,0.671321,0.547196
398,2013-08-09,Wind Farm 8,0.748910,0.596867
